In [8]:
import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\mocha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\mocha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\mocha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\mocha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\mocha\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [9]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data/intents.json').read()
intents = json.loads(data_file)

# Check keys and firs five data in key value

In [11]:
intents.keys()

dict_keys(['intents'])

In [12]:
intents['intents'][:5]

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'context_set': ''},
 {'tag': 'creator',
  'patterns': ['what is the name of your developers',
   'what is the name of your creators',
   'what is the name of the developers',
   'what is the name of the creators',
   'who created you',
   'your developers',
   'your creators',
   'who are your developers',
   'developers',
   'you are made by',
   'you are made by whom',
   'who

#  Preprocess data
Before creating a machine learning or deep learning model when working with text data, there are a number of preprocessing steps that must be taken. We need to preprocess the data using a variety of processes depending on the needs.

The simplest and initial action you can do with text data is to tokenize it. The method of tokenizing involves dividing the entire text into smaller units, such as words.

Using the nltk.word_tokenize() method, we tokenize the phrase in this case and attach each word from the words list after iterating over the patterns. For our tags, we also make a list of classes.

In [13]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

We will now lemmatize each word and eliminate any repetitions from the list. Lemmatizing entails taking a word and changing it into a lemma. After that, a pickle file is made to house the Python objects that will be used for prediction.

In [14]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
#If you save this two list comment out this code and run again


OSError: No such file or directory: 'C:\\Users\\mocha\\AppData\\Roaming\\nltk_data\\corpora\\wordnet\\lexnames'

# > If you save this two list comment out this code and run again

In [ ]:
# pickle.dump(words,open('words.pkl','wb'))
# pickle.dump(classes,open('classes.pkl','wb'))

# Create training and testing data
Now that the training data has been created, the input and output will be included. The pattern will serve as our input, and the class to which the pattern belongs will serve as our output. However, since the machine cannot interpret language, we must convert it to numbers.

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# Build the model

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
81/81 [==============================] - 1s 2ms/step - loss: 3.6075 - accuracy: 0.0395
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.4209 - accuracy: 0.1309
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.2103 - accuracy: 0.1926
Epoch 4/200
81/81 [==============================] - 0s 3ms/step - loss: 2.9198 - accuracy: 0.2247
Epoch 5/200
81/81 [==============================] - 0s 3ms/step - loss: 2.6930 - accuracy: 0.2938
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 2.4807 - accuracy: 0.3407
Epoch 7/200
81/81 [==============================] - 0s 3ms/step - loss: 2.2439 - accuracy: 0.3901
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.0214 - accuracy: 0.4593
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 1.9467 - accuracy: 0.4667
Epoch 10/200
81/81 [==============================] - 0s 2ms/step - loss: 1.7577 - accuracy: 0.5407
Epoch 11/

> If You save the model then comment out the cell bellow and run this

In [ ]:
# model.save('chatbot_model.h5', hist)
# print("model created")

# Predict the response

> source: https://data-flair.training/blogs/python-chatbot-project/
> follow this link and get graphical unit code. ad test it
# Thank You